In [1]:
import hail as hl
hl.init(app_name="make_finalised_mt")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eriurn/submit/python37_intel/lib/python3.7/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.0.0-78/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-02-04 09:07:18 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-02-04 09:07:19 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-02-04 09:07:20 WARN  Client:69 - Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2022-02-04 09:07:30 WARN  DataStreamer:1683 - Abandoning BP-1268015869-192.168.11.216-1641416617205:blk_1092238597_18498148
2022-02-04 09:07:30 WARN  DataStreamer:1688 - Excluding datanode DatanodeInfoWithStorage[10.0.1.97:50010,DS-f7d113f5-c3b3-4b27-82c3-1e0cb11a4555,DISK]
2022-02-04 09:07:31 WARN  DataStreamer:1683 - Abandoning BP-1268015869-192.168.11.216-1641416617205:blk_1092238599_18498150
2022-02-04 09:07:31 WARN  DataStreamer:1688 - Excluding datanode DatanodeInfoWithStorage[10.0.1.97:50010,DS-f7d113f5-c3b3-4b27-82c3-1e0cb11a4555,DISK]


Running on Apache Spark version 3.1.2
SparkUI available at http://dice01-mlx.mlx:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.68-13190f0b6103
LOGGING: writing to /home/eriurn/Hail-joint-caller/notebooks/hail-20220204-0907-0.2.68-13190f0b6103.log


In [4]:
# Some variables
path_to_mt = "file:///home/eriurn/Hail-joint-caller/data/reblock_key_alleles.mt"
path_to_vqsr_ht = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/VQSR.filtered.final.ht"
out_mt_path = "file:////directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/vqsr_finalised.mt"

In [5]:
def annotate_vqsr(mt, vqsr_ht):
    mt = mt.annotate_rows(**vqsr_ht[mt.row_key])
    
    mt = mt.annotate_rows(info=vqsr_ht[mt.row_key].info)
    
    mt = mt.annotate_rows(
        filters=mt.filters.union(vqsr_ht[mt.row_key].filters),
    )
    
    mt = mt.annotate_globals(**vqsr_ht.index_globals())
    
    return mt

In [6]:
mt = hl.read_matrix_table(path_to_mt)

In [7]:
vqsr_ht = hl.read_table(path_to_vqsr_ht)
mt = annotate_vqsr(mt, vqsr_ht)

In [9]:
for ht in [vqsr_ht]:
    mt = mt.annotate_globals(**ht.index_globals())

In [10]:
mt.write(out_mt_path, overwrite=True)

2022-02-04 10:47:36 Hail: INFO: wrote matrix table with 58918284 rows and 5 columns in 2586 partitions to file:////directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/vqsr_finalised.mt
    Total size: 3.58 GiB
    * Rows/entries: 3.58 GiB
    * Columns: 51.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  82032 rows (11.73 MiB)
